<img src="https://raw.githubusercontent.com/probcomp/notebook/e66a399561d069a5dfe19c7efd8dd62c09a80787/tutorials/resources/header.png"/>
<img src="https://www.yammer.com/api/v1/uploaded_files/61754420/preview/CMI_rgb_horizontal_logo.png?_t=1525896635778" style='height:75px'/>

# Exploratory data analysis with BayesDB for child psychology


Authored by: Ulrich Schaechtle, Veronica Sara Weiner of the MIT Probabilistic Computing Project (Probcomp) with Arno Klein and Jon Clucas of the Child Mind Institute MATTER Lab.

In this notebook, we will use BayesDB for exploratory data analysis of the questionnaire responses data set provided by the [Child Mind Institute](https://childmind.org/). The data set contains individual question-items of a [set of
questionnaires](http://fcon_1000.projects.nitrc.org/indi/cmi_healthy_brain_network/assessments/master-list.html) related to childhood mental health and a set of structural MRI features. 

This notebook will cover the following topics:

1. List of questionnaires and links to individual questions
2. Analysis with BayesDB
3. Exploring probable dependencies between variables and comparing CrossCat dependence probability to linear correlation
4. Taking a closer look at dependencies and the models built by CrossCat
5. Aggregating across an ensemble of CrossCat models to assess the similarity of subjects

## 1. List of questionnaires and links to individual questions

The following are the questionnares from which data has been collected by the Child Mind Institute, and that we analyze in this notebook. Each link leads to a set of questions. We analyze the data in the original questionnaire format (as categorical responses to individual questions). Compound/summary scores are not included, the diagnoses have been transformed into individual binary variables.
   
- [ACE: Adverse Childhood Experience (ACE)](resources/html/ACE.html)
- [APQ_P: Alabama Parenting Questionnaire (APQ) Parent-Report](resources/html/APQ_P.html)
- [APQ_S: Alabama Parenting Questionnaire (APQ) Self-Report](resources/html/APQ_S.html)
- [ARI_P: Affective Reactivity Index (ARI) Parent-Report](resources/html/ARI_P.html)
- [ARI_S: Affective Reactivity Index (ARI) Self-Report](resources/html/ARI_S.html)
- [ASR: Adult Self Report](resources/html/ASR.html)
- [ASSQ: Autism Spectrum Screening Questionnaire (ASSQ)](resources/html/ASSQ.html)
- [C3SR (Conners-SR): Conners 3 - Self-Report (C3SR)](resources/html/C3SR.html)
- [CAARS: Conners' Adult ADHD Rating Scales (CAARS)](resources/html/CAARS.html)
- [CBCL: Child Behavior Checklist (CBCL) -- Age 6-18](resources/html/CBCL.html)
- [CBCL_Pre: Child Behavior Checklist (CBCL) Pre-School](resources/html/CBCL_Pre.html)
- [CCSC: Children's Coping Strategies Checklist-Revised](resources/html/CCSC.html)
- [CDI2SR: Children's Depression Index (CDI) Self Report](resources/html/CDI2SR.html)
- [CDI2_P: Children's Depression Index (CDI) Parent Report](resources/html/CDI2_P.html)
- [CIS_P: Columbia Impairment Scale (CIS) Parent Report](resources/html/CIS_P.html)
- [CIS_SR: Columbia Impairment Scale - Self-Report (CIS-SR)](resources/html/CIS_SR.html)
- [CPIC: Children's Perception of Interparental Conflict Scale (CPIC)](resources/html/CPIC.html)
- [DTS: Distress Tolerance Scale (DTS)](resources/html/DTS.html)
- ~~[IAT: Internet Addiction Test (IAT)](resources/html/IAT.html)~~
- [ICU_P: Inventory of Callous-Unemotional Traits (ICU) Parent Report](resources/html/ICU_P.html)
- [ICU_SR: Inventory of Callous-Unemotional Traits (ICU) Self Report](resources/html/ICU_SR.html)
- [MFQ_P: Mood and Feelings Questionnaire (MFQ) Parent Report](resources/html/MFQ_P.html)
- [MFQ_SR: Mood and Feelings Questionnaire (MFQ) Self Report](resources/html/MFQ_SR.html)
- ~~[PCIAT: Parent-Child Internet Addiction Test](resources/html/PCIAT.html)~~
- [PSI: Parenting Stress Index (PSI)](resources/html/PSI.html)
- [SAS: Socal Aptititude Scale (SAS)](resources/html/SAS.html)
- [SCARED_P: Screen for Child Anxiety Related Disorders (SCARED) Parent Report](resources/html/SCARED_P.html)
- [SCARED_SR: Screen for Anxiety Related Disorders (SCARED) Self Report](resources/html/SCARED_SR.html)
- [SCQ: Social Communication Questionnaire](resources/html/SCQ.html)
- [SDQ: Strength and Difficulties Questionnaire](resources/html/SDQ.html)
- [SDS: Sleep Disturbance Scale (SDS)](resources/html/SDS.html)
- [SRS_Preschool: Social Responsiveness Scale (SRS) Preschool](resources/html/SRS_Preschool.html)
- [SRS_School: Social Responsiveness Scale (SRS) School Age](resources/html/SRS_School.html)
- [STAI: State-Trait Anxiety Inventory for Adults](resources/html/STAI.html)
- [SWAN : The SWAN Rating Scale for ADHD](resources/html/SWAN .html)
- [Symptom_Checklist: Symptom Checklist - Parent](resources/html/Symptom_Checklist.html)
- [TRF_Preschool_Age: Teacher Report Form (TRF) Preschool Age](resources/html/TRF_Preschool_Age.html)
- [TRF_School_Age: Teacher Report Form (TRF) School Age](resources/html/TRF_School_Age.html)
- [WHODAS_P: WHO Disability Assessment Schedule (WHODAS) Parent-Report](resources/html/WHODAS_P.html)
- [WHODAS_SR: WHO Disability Assessment Schedule (WHODAS) Self-Report](resources/html/WHODAS_SR.html)
- [YFAS: Yale Food Addiction Scale (YFAS)](resources/html/YFAS.html)
- [YSR: Youth Self Report (YSR)](resources/html/YSR.html)



## 2. Analysis with BayesDB

### 2a. Setting up the Jupyter environment

The first step is to load the `jupyter_probcomp.magics` library, which provides BayesDB hooks for data exploration, plotting, querying, and analysis through this Jupyter notebook environment. The second cell allows plots from matplotlib and javascript to be shown inline.

In [ ]:
%load_ext jupyter_probcomp.magics

In [ ]:
%matplotlib inline
%vizgpm inline

### 2b. Creating a BayesDB `.bdb` file on disk

We next use the `%bayesdb` magic to create a `.bdb` file on disk named `childmind_smri_questions.bdb`. This file will store all the data and models created in this session.

In [ ]:
!rm -f resources/bdb/childmind_smri_questions.bdb
%bayesdb resources/bdb/childmind_smri_questions.bdb
bdb = %get_bdb
import os
os.environ['LOOM_VERBOSITY'] = '0'
import bayeslite
from bayeslite.backends.loom_backend import LoomBackend
from bayeslite import bayesdb_register_backend
bayesdb_register_backend(bdb, LoomBackend(os.path.abspath('loom_files/')))

### 2c. Ingesting data from a `.csv` file into a BayesDB table

The questionnaire dataset is stored in the csv file `resources/init_data.csv`. Each column of the csv file is a variable, and each row is a record. We use the `CREATE TABLE` BQL query, with the pathname of the csv file, to convert the csv data into a database table named `raw_questionnaire_responses`.

In [ ]:
%bql CREATE TABLE "raw_questionnaire_responses" FROM 'resources/data/smri_questions.csv'

Almost all datasets have missing values, and special tokens such as `NaN` or `NA` indicating a particular cell is missing. In the questionnaire data, empty strings are used. To tell BayesDB to treat empty strings as SQL `NULL` we use the `.nullify` command, followed by the name of the table and the string `''` which represents missing data. Over 250,000 cells have been converted to `NULL`, illustrating that the data is quite sparse.

In [ ]:
%bql .nullify raw_questionnaire_responses ''

### 2d. Running basic queries on the table using BQL and SQL

Now that the questionnaire dataset has been loaded into at table, and missing values converted to `NULL`, we can run standard SQL queries to explore the contents of the data. For example, we can select the first 5 records. Observe that each row in the table is a particular country, and each column is a macreconomic variable. Scroll through the names in the header of the table to get a sense of the marcoeconomic variables in the dataset. 

In [ ]:
%bql SELECT * FROM "raw_questionnaire_responses" LIMIT 5;

We can also find the total number of records (i.e. subjects).

In [ ]:
%bql SELECT COUNT(*) as N FROM "raw_questionnaire_responses";

### 2e. Creating a BayesDB population for the questionnaire response data

The notion of a "population" is a central concept in BayesDB. For a standard database table, such as `raw_questionnaire_responses`, each column is associated with a [data type](https://sqlite.org/datatype3.html), which in sqlite3 are `TEXT`, `REAL`, `INTEGER`, and `BLOB`. For a BayesDB population, each variable is associated with a _statistical data type_. These statistical types, such as `NOMINAL`, `NUMERICAL`, `MAGNITUDE`, and `COUNTS`, specify the set of values and default probability distributions used for building probabilistic models of the data in the population. In this tutorial, we will use the `NUMERICAL` and `NOMINAL` statistical data types.

We can use the `GUESS SCHEMA FOR <table>` command from the Metamodeling Language (MML) in BayesDB to guess the statistical data types of variables in the table. The guesses use heuristics based on the contents in the cells. The `num_distinct` column shows the number of unique values for that variable, and the `reason` column explains which heuristic was used to make the guess.

In [ ]:
%mml GUESS SCHEMA FOR "raw_questionnaire_responses"

In [ ]:
%%mml
CREATE POPULATION "questionnaire_responses_population" FOR "raw_questionnaire_responses" WITH SCHEMA (
    GUESS STATTYPES OF (*);
    -- stuff that the guess suggested to ignore:
    SET STATTYPES OF
         "SCQ_30",
         "SCQ_01",
         "SCQ_28",
         "ADHD",
         "Anxiety",
         "Autism",
         "Other"
    TO
        NOMINAL;
    SET STATTYPE OF 
         "Age" 
    TO
        NUMERICAL;
    IGNORE	 
         "EID";
);

### 2f. Creating initial multivariate models of the data

#### Turn on multi-core computing with BayesDB

In [ ]:
%multiprocess on

This way, we can use all 64 cores of this machine in parallel. 

Now that we have created the `questionnaire_responses_population` population, the next step is to analyze the data by building probabilistic models which explain the data generating process. Probabilistic data analyses in BayesDB are specified using an `MODELING SCHEMA`. The default model discovery engine in BayesDB is Cross-Categorization [(Crosscat)](http://jmlr.org/papers/v17/11-392.html). CrossCat is a Bayesian factorial mixture model which learns a full joint distribution over all variables in the population, using a divide-and-conquer approach. We will explore CrossCat more in this notebook.

For now we use MML to declare the an analysis schema named `questionnaire_responses_m` for the `questionnaire_responses_population` population.

In [ ]:
%%mml
CREATE GENERATOR FOR "questionnaire_responses_population" USING loom;

After creating the generator, we now need to initialize `MODELS` for the schema. We can think of a `MODEL` as specifying a hypothesis space of explanations for the data generating process for the population, and each `ANALYSIS` is a candidate hypothesis. We start by creating only 50 models, which are initialized __randomly__.

In [ ]:
%mml INITIALIZE 50 MODELS IF NOT EXISTS FOR "questionnaire_responses_population";

Next, we run analysis for four hours.

In [ ]:
%mml ANALYZE "questionnaire_responses_m" FOR 240 MINUTES WAIT (OPTIMIZED);

## 3. Exploring probable dependencies between variables and comparing CrossCat dependence probability to linear correlation

As mentioned earlier, all BQL queries are aggregated across the 60 analyses in the ensemble. We will create a table named `dependencies` which contains the pairwise `DEPENDENCE PROBABILITY` values between the questionnaire variables. The value of a cell (between 0 and 1) is the fraction of analyses in the ensemble where those two variables are detected to be probably dependent (i.e. they are in the same view).

In [ ]:
%%bql
CREATE TABLE dependencies AS
    ESTIMATE DEPENDENCE PROBABILITY AS "depprob"
        FROM PAIRWISE VARIABLES OF questionnaire_responses_population;

We again summarize the `dependencies` table using a heatmap. Study this dependence heatmap, and compare it to the heatmap produced when there was only 1 analysis. Which common-sense dependencies were missed by the single model, but identified by the ensemble as probably dependent?

Find a full list of questionnaire items [here.](questionnaires-items.ipynb)

In [ ]:
%bql .interactive_heatmap SELECT name0, name1, depprob FROM dependencies;

Let us compare dependence probabilities from CrossCat to linear correlation values, a very common technique for finding predictive relationships. We can compute the (and its p-value) in BayesDB using the `CORRELATION` and `CORRELATION PVALUE` queries. The following cell creates a table named `correlations`, which contains the R and p-value for all pairs of variables.

In [ ]:
%%bql
CREATE TABLE "correlations" AS
ESTIMATE
    CORRELATION AS "correlation",
    CORRELATION PVALUE AS "pvalue"
FROM PAIRWISE VARIABLES OF "questionnaire_responses_population"

__Emphasis__: There is a signficiant difference between `DEPENDENCE PROBABILITY`, `CORRELATION`, and `CORRELATION PVALUE`. We outline these differences below, which will help us make comparisons between predictive relationships detected by CrossCat versus Pearson correlation.

- `DEPENDENCE PROBABILITY`: Returns a value between [0,1] indicating the __probability there exists__ a predictive relationship (statistical dependence) between two variables.

- `CORRELATION`: Returns a value between [0,1] indicating the __strength__ of the linear relationsip between two variables, where 0 means no linear correlation, and 1 means perfect linear correlation.

- `CORRELATION PVALUE`: Returns a value between (0, 1) indicating the tail probability of the observed correlation value between two variables, under the null hypothesis that the two variables have zero correlation.

Based on these distinctions, there is no immediate way to numerically compare `DEPENDENCE PROBABILITY` with `CORRELATION/CORRELATION PVALUE`. However, it is possible to compare the inferences about predictive relationships that each method gives rise to, which we do in the next section.

Let us first produce a heatmap of the raw correlation values. The following query shows the raw correlation values (between 0 and 1) for all pairs of variables where the p-value is less than 0.01 (note that we are not accounting for multiple-testing using e.g. Bonferroni correction). Pairs of variables where the p-value exceeds 0.01 (and thus the null hypothesis of independence cannot be rejected) are shown in gray. The sparsity of the data makes it difficult to draw inferences about many variables.

In [ ]:
%bql .interactive_heatmap SELECT name0, name1, "correlation" FROM "correlations" WHERE "pvalue" < 0.1

Explore the heatmap, and compare it to the heatmap from `DEPENDENCE PROBABILITY`. The patterns of dependence relationships differ significantly, how?

We can use BQL to find variables which CrossCat believes are probably dependent, but correlation believes are independent (either the null hypothesis of independence cannot be rejected, or the correlation value is significant and near zero).

In [ ]:
%%sql
SELECT
    "name0",
    "name1",
    "dependencies"."depprob",
    "correlations"."correlation",
    "correlations"."pvalue"
FROM
    "dependencies"
    JOIN "correlations"
    USING ("name0", "name1")
WHERE
    -- CrossCat: probability dependent.
    "dependencies"."depprob" > 0.85
    AND (
    -- Correlation: cannot reject null hypothesis of independence.
    "correlations"."pvalue" > 0.05
    OR (
    -- Correlation: linear relationship is significant and near zero.
    "correlations"."pvalue" < 0.05 AND "correlations"."correlation" < 0.05))

#### Quesstionnaire items
- SWAN_06: engages in tasks that require sustained mental effort.
- SDQ_25: Good attention span, sees chores or homework through to the end.
- IAT_13: How often do you snap, yell, or act annoyed if someone bothers you while you are online

Find a full list of questionnaire items [here.](questionnaires-items.ipynb)

#### Example of no probability of dependence being high

In [ ]:
%bql .show_histograms --table raw_questionnaire_responses --column1 SDQ_25 --column2 Sex

We can use also BQL to find variables which CrossCat believes are probably independent, but correlation believes are dependent (a statistically significant non-zero correlation value, where we are using an R cutoff of 0.15). The following query shows a list of such variables.

In [ ]:
%%sql
SELECT
    "name0",
    "name1",
    "dependencies"."depprob",
    "correlations"."correlation",
    "correlations"."pvalue"
FROM
    "dependencies"
    JOIN "correlations"
    USING ("name0", "name1")
WHERE
    -- CrossCat: high uncertainty about dependence probability.
    "dependencies"."depprob" < 0.05
    AND (
    -- Correlation: statistically significant dependence.
    "correlations"."pvalue" < 0.05 AND "correlations"."correlation" > 0.15)
LIMIT 10

The top correlations with 0 probability of dependency are all taken from the
Alabama Parenting Questionnaire (parent report):

- APQ_P_11: You help your child with his/her homework
- APQ_P_18: You hug or kiss your child when he/she has done something well
- APQ_P_28: You don't check that your child comes home at the time he/she was supposed to
- APQ_P_30: You don't check that your child comes home at the time he/she was supposed to
- APQ_P_40: You calmly explain to your child why his/her behavior was wrong when he/she misbehaves
- ...


Find all the questions of all questionnaires [here.](questionnaires-items.ipynb)

#### Example of probability of dependence being 0

In [ ]:
%bql .show_histograms --table raw_questionnaire_responses --column1 APQ_P_28 --column2 Sex

#### Quesstionnaire item
- APQ_P_28: You don't check that your child comes home at the time he/she was supposed to

## 4. Taking a closer look at dependencies and the CrossCat state

### 4a. Listing variables probably dependent with autism diagnosis, ADHD diagnosis, anxiety diagnosis and age


In [ ]:
%%bql
SELECT * FROM "dependencies"
    WHERE name0 = 'Autism'
    ORDER BY depprob DESC
    LIMIT 50

In [ ]:
%%bql
SELECT * FROM "dependencies"
    WHERE name0 = 'ADHD'
    ORDER BY depprob DESC
    LIMIT 50

In [ ]:
%%bql
SELECT * FROM "dependencies"
    WHERE name0 = 'Anxiety'
    ORDER BY depprob DESC
    LIMIT 50

In [ ]:
%%bql
SELECT * FROM "dependencies"
    WHERE name0 = 'age'
    ORDER BY depprob DESC
    LIMIT 10

Alabama Parenting Questionnaire (parent report)

- apq_p_06:   You child fails to leave a note to let you know where he/she is going
- apq_p_10:   Your child stays out in the evening past the time that he/she is supposed to be home
- apq_p_11:   You help your child with his/her homework
- apq_p_19:   Your child goed out with a set time to be home
- apq_p_21:   Your child goes out after dark without an adult
- apq_p_30:  Your child comes home from school more than an hour past the time you expect him/her to be home
- apq_p_32:  Your child is at home without adult supervision


Alabama Parenting Questionnaire (self report)

- apq_sr_06:  You fail to leave a note or let your parents know where you are going
- apq_sr_10:  You stay out in the evening past the time you are supposed to be home

### 4b. Visualizing CrossCat states
For simplicity, we now create a new, smaller population with the 110 variables above. We
analyze again (which is much quicker now, because we have only a fraction of the column)
and plot the individual crosscat states.

The full `raw_questionnaire_responses` table contains over 700 columns. In this notebook, our exploratory analysis will be based on a random subsample of 100 columns. To create the subsample, we use the `.subsample_columns` magic. The `--keep` flag accepts a list of column names which should be kept. We will keep the columns we found in the three tables above. Finally, `raw_questionnaire_responses_subsample` is the name of the new table, and 110 is the number of columns to downsample to.

In [ ]:
%bql .subsample_columns  raw_questionnaire_responses0 raw_questionnaire_responses_subsample 110 --seed=1 \
--keep EID Autism_Spectrum_Disorder Attention_Deficit_Hyperactivity_Disorder assq_03 assq_04 assq_09 assq_10 assq_11 assq_13 assq_14 assq_15 assq_16 assq_18 assq_19 assq_20 assq_21 assq_22 assq_23 assq_24 assq_26 assq_27 scq_03 scq_04 scq_06 scq_07 scq_08 scq_11 scq_12 scq_13 scq_14 scq_15 scq_16 assq_06 assq_07 assq_08 scq_09 scq_38 assq_17 scq_05 scq_19 assq_02 assq_05 assq_25 scared_p_08 scq_18 scq_33 assq_12 sdq_23 sdq_11 scq_17 scq_10 assq_01 sex sdq_02 sdq_10 sdq_21 swan_03 swan_10 swan_11 swan_12 swan_13 swan_14 swan_15 swan_16 swan_17 swan_18 sdq_15 sdq_25 swan_01 swan_02 swan_04 swan_05 swan_06 swan_07 swan_08 swan_09 psi_32 sdq_26 mfq_p_21 mfq_p_07 sas_01 sas_06 sas_07 sds_03 psi_29 psi_36 sas_02 sas_04 sas_05 sas_10 apq_p_37 ari_p_01 ari_p_02 ari_p_03 ari_p_05 ari_p_06 ari_p_07 mfq_p_11 psi_17 psi_28 age apq_p_06 apq_p_10 apq_p_11 apq_p_19 apq_p_21 apq_p_30 apq_p_32 apq_sr_06 apq_sr_10

In [ ]:
%%mml
CREATE POPULATION "questionnaire_responses_subsample_population" 
    FOR "raw_questionnaire_responses_subsample" WITH SCHEMA (
        GUESS STATTYPES FOR (*);
    );

In [ ]:
%multiprocess on

In [ ]:
%mml CREATE ANALYSIS SCHEMA "questionnaire_responses_subsample_m" FOR "questionnaire_responses_subsample_population" WITH BASELINE crosscat();
%mml INITIALIZE 60 MODELS FOR "questionnaire_responses_subsample_m";

Analysis is much quicker now since we are looking only at a fraction of all the columns.


In [ ]:
%mml ANALYZE "questionnaire_responses_subsample_m" FOR 10 MINUTES WAIT (OPTIMIZED);

#### Visualizing a CrossCat hypothesis

As mentioned earlier, CrossCat learns the full joint distribution of all variables in the population using divide-and-conquer:

- First, CrossCat partitions the variables into a set of _views_; all the variables in a particular view are modeled jointly, and two variables in different views are independent of one another.
- Second, within each view, CrossCat clusters the rows using a non-parametric mixture model.

The name Cross-Categorization is derived from this two-step process: first categorize the variables into views, and then categorize the rows into clusters within each view of variables. It is important to note that two different views A and B are likely to induce different clusterings of the rows.

To get a sense of CrossCat's hypothesis space, we can render the hypothesis specified by a particular analysis using the `.render_crosscat [options] <analysis_schema> <analysis_identifier>` plotting command. The `--subsample=50` option says to only show a subsample of 50 rows in the rendering (even though `questionnaire_responses_subsample_m` is modeling all countries in the `childmind` population); `--rowlabels=country` specifies which column in the table to use to label the rows. Finally `questionnaire_responses_subsample_m 0` means to render the first (and only) analysis in the `questionnaire_responses_subsample_m` anlaysis schema.

__To view a full-size image of the rendering, either double click the image, or right-click and select "Open image in new tab."__

In [ ]:
%mml .render_crosscat \
    --subsample=50 --rowlabels=EID --xticklabelsize=small --yticklabelsize=xx-small --progress=True --width=64 \
    questionnaire_responses_subsample_m 0

## 5. Similarity of subjects in different contexts

In the heatmap below, each row and column is a subject, and the value of a cell (between 0
and 1) indicates the probability that those two subject are relevant for formulating
predictions about each other. Do these clusters of subjects make sense?


### 5a. Similarity with respect to diagnosis: autism

In [ ]:
%%bql
CREATE TABLE "similarity_autism" AS
    ESTIMATE SIMILARITY IN THE CONTEXT OF "Autism"
    FROM PAIRWISE "questionnaire_responses_population";

In [ ]:
%bql .interactive_heatmap --label0=rowid --label1=EID --table=raw_questionnaire_responses\
    SELECT * FROM "similarity_autism"

### 5b. Similarity with respect to diagnosis: ADHD

In [ ]:
%%bql
CREATE TABLE "similarity_adhd" AS
    ESTIMATE SIMILARITY IN THE CONTEXT OF "ADHD"
    FROM PAIRWISE "questionnaire_responses_population";

In [ ]:
%bql .interactive_heatmap --label0=rowid --label1=EID --table=raw_questionnaire_responses \
    SELECT * FROM "similarity_adhd"

### 5c. Similarity with respect to a specific item in autism screening questionnaire
- ASSQ_6 (Autism Spectrum Screening Questionnaire): Child has a deviant style of communication with a formal, fussy, old-fashioned or robotlike language (i.e. talks differently than other children, in a formal or stilted way)

In [ ]:
%%bql
CREATE TABLE "similarity_assq_06" AS
    ESTIMATE SIMILARITY IN THE CONTEXT OF "ASSQ_06"
    FROM PAIRWISE "questionnaire_responses_population";

In [ ]:
%bql .interactive_heatmap --label0=rowid --label1=EID --table=raw_questionnaire_responses \
    SELECT * FROM "similarity_assq_06"